<a href="https://colab.research.google.com/github/Black2oo4g/GoogleColab/blob/main/DATAACOMEMOP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
from google.colab import drive
import pandas as pd
import json

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Asegurarse que columnas estén bien
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Definir meses y su número
meses_mapping = {
    "ENERO": "01",
    "FEBRERO": "02",
    "MARZO": "03",
    "ABRIL": "04",
    "MAYO": "05",
    "JUNIO": "06",
    "JULIO": "07",
    "AGOSTO": "08",
    "SEPTIEMBRE": "09",
    "OCTUBRE": "10",
    "NOVIEMBRE": "11",
    "DICIEMBRE": "12"
}

# 8. Crear JSON
cols = df_energy.columns.tolist()
json_list = []

for _, row in df_energy.iterrows():
    cod = row["ID"]
    if pd.isna(cod):
        continue
    try:
        cod_int = int(cod)
    except (ValueError, TypeError):
        continue
    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre, mes_num in meses_mapping.items():
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date):
                    continue
                venc_date = emi_date + pd.Timedelta(days=30)

                # El año lo sacamos de emi_date
                year = emi_date.strftime("%Y")

                # El mes es basado en el nombre de la columna
                periodo = f"{year}-{mes_num}"

                json_list.append({
                    "cod": cod_int,
                    "periodo": periodo,
                    "monto": float(monto) if pd.notna(monto) else 0.0,
                    "emi": emi_date.strftime("%Y-%m-%d"),
                    "venc": venc_date.strftime("%Y-%m-%d"),
                    "operacion": str(operacion)
                })

# 9. Guardar JSON
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/salida_pagos.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 10. Guardar socios no reconocidos
output_unrecognized_path = "/content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos.xlsx"
df_unrecognized.to_excel(output_unrecognized_path, index=False)

# 11. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo de socios no reconocidos guardado en: {output_unrecognized_path}")

# 12. Mostrar ejemplo de JSON generado
print(json.dumps(json_list[:5], indent=4, ensure_ascii=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total registros RECONOCIDOS (con ID): 254
Total registros NO RECONOCIDOS (sin ID): 169
Archivo de socios no reconocidos guardado en: /content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos.xlsx
[
    {
        "cod": 529,
        "periodo": "2025-01",
        "monto": 116.6,
        "emi": "2025-04-03",
        "venc": "2025-05-03",
        "operacion": "17399"
    },
    {
        "cod": 529,
        "periodo": "2025-02",
        "monto": 120.8,
        "emi": "2025-03-26",
        "venc": "2025-04-25",
        "operacion": "17709"
    },
    {
        "cod": 529,
        "periodo": "2025-03",
        "monto": 135.6,
        "emi": "2025-04-23",
        "venc": "2025-05-23",
        "operacion": "18106.0"
    },
    {
        "cod": 18,
        "periodo": "2025-01",
        "monto": 34.5,
        "emi": "2025-03-03",
        "venc": "2025-04-02",
       

In [ ]:
from google.colab import drive
import pandas as pd
import json

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrames de reconocidos y no reconocidos
df_recognized = df_energy.dropna(subset=["ID"])
df_unrecognized = df_energy[df_energy["ID"].isna()]

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]
meses_mapping = {mes: str(i+1).zfill(2) for i, mes in enumerate(meses)}

# 9. Crear JSON agrupado por cod
cols = df_energy.columns.tolist()
json_dict = {}

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    first_emi_found = False
    base_year = None
    mes_idx = 0

    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue

                # Convertir la fecha de pago directamente sin cálculos
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date) or pd.isna(monto):
                    continue

                # Definir periodo
                periodo = f"{emi_date.year}-{str(mes_idx + 1).zfill(2)}"

                pago = {
                    "periodo": periodo,
                    "monto": float(monto),
                    "fecha_pago": emi_date.strftime("%Y-%m-%d"),  # Usar la fecha de pago directamente
                    "operacion": str(operacion)
                }

                if cod not in json_dict:
                    json_dict[cod] = []

                json_dict[cod].append(pago)
                mes_idx += 1

# 10. Guardar JSON agrupado
import os
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/energia-2025.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_dict, f, indent=4, ensure_ascii=False)

# 11. Guardar socios no reconocidos
output_unrecognized_path = "/content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos-2025.xlsx"
df_unrecognized.to_excel(output_unrecognized_path, index=False)

# 12. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo actualizado: {energy_path}")
print(f"Archivo de socios no reconocidos: {output_unrecognized_path}")

# 13. Mostrar un ejemplo del JSON generado (primer comerciante)
first_cod = next(iter(json_dict))
print(f"Ejemplo de pagos para el COD {first_cod}:")
print(json.dumps(json_dict[first_cod], indent=4, ensure_ascii=False))


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Total registros RECONOCIDOS (con ID): 395
Total registros NO RECONOCIDOS (sin ID): 28
Archivo actualizado: /content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx
Archivo de socios no reconocidos: /content/drive/MyDrive/AcomemopV/Data/socios_no_reconocidos-2025.xlsx
Ejemplo de pagos para el COD 529:
[
    {
        "periodo": "2025-01",
        "monto": 116.6,
        "fecha_pago": "2025-04-03",
        "operacion": "17399"
    },
    {
        "periodo": "2025-02",
        "monto": 120.8,
        "fecha_pago": "2025-03-26",
        "operacion": "17709"
    },
    {
        "periodo": "2025-03",
        "monto": 135.6,
        "fecha_pago": "2025-04-23",
        "operacion": "18106.0"
    }
]


In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]

# 9. Crear JSON de pagos para todos los comerciantes (sin agrupar)
json_list = []

cols = df_energy.columns.tolist()
for _, row in df_recognized.iterrows():
    cod = int(row["ID"])

    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue

                # Convertir la fecha de pago directamente sin cálculos
                emi_date = pd.to_datetime(emi_raw, dayfirst=True, errors="coerce")
                if pd.isna(emi_date) or pd.isna(monto):
                    continue

                # Definir periodo
                #periodo = f"{emi_date.year}-{str(mes_idx + 1).zfill(2)}"
                mes_num = str(meses.index(mes_nombre) + 1).zfill(2)
                periodo = f"2025-{mes_num}"


                pago = {
                    "id_comerciante": cod,
                    "periodo": periodo,
                    "monto": float(monto),
                    "fecha_pago": emi_date.strftime("%Y-%m-%d"),  # Usar la fecha de pago directamente
                    "operacion": str(operacion)
                }

                json_list.append(pago)


# 10. Guardar JSON (sin agrupar por ID)
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/energia-2025.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 11. Mostrar solo los primeros 5 registros
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 12. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
#print("Total registros NO RECONOCIDOS (sin ID):", len(df_unrecognized))
print(f"Archivo actualizado: {energy_path}")
#print(f"Archivo de socios no reconocidos: {output_unrecognized_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2025-01",
        "monto": 116.6,
        "fecha_pago": "2025-04-03",
        "operacion": "17399"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-02",
        "monto": 120.8,
        "fecha_pago": "2025-03-26",
        "operacion": "17709"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-03",
        "monto": 135.6,
        "fecha_pago": "2025-04-23",
        "operacion": "18106"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-01",
        "monto": 34.5,
        "fecha_pago": "2025-03-03",
        "operacion": "17371"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-02",
        "monto": 31.9,
        "fecha_pago": "2025-03-27",
        "operacion": "17711"
    },
    {
        "id_comerc

In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS2024", header=0)
df_target = pd.read_excel(energy_path, sheet_name="PAGOS2022", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df["MANZANA"] = df["MANZANA"].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref["MANZANA"] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target["MANZANA"] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021_actualizado.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS2024", index=False)
    df_target.to_excel(writer, sheet_name="PAGOS2022", index=False)

# 10. Verificación
print("IDs asignados en PAGOS2022:")
print(df_target[["SOCIO", "MANZANA", df_target.columns[0], "ID"]].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-13-f5e0da5a9241>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
424    30.0
425    30.0
426     NAN
427     NAN
428     NAN
Name: Unnamed: 0, Length: 429, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)
<ipython-input-13-f5e0da5a9241>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
424    30.0
425    30.0
426     NAN
427     NAN
428    13.0
Name: , Length: 429, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Pri

IDs asignados en PAGOS2022:
                               SOCIO MANZANA          ID
0          SILVA SALDARRIAGA ALBERTO      12  1.0  529.0
1          CARRASCO CASTILLO ALFONSO     13A  1.0   18.0
2          VILLEGAS PACHERREZ MANUEL     13B  1.0  621.0
3  HIDALGO VDA. DE CHAPARRO HILBERTA    14-A  1.0   19.0
4  HIDALGO VDA. DE CHAPARRO HILBERTA    14-B  1.0  655.0


In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS2024", header=0)
df_target = pd.read_excel(energy_path, sheet_name="PAGOS2022", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df["MANZANA"] = df["MANZANA"].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref["MANZANA"] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target["MANZANA"] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA2021_actualizado.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS2024", index=False)
    df_target.to_excel(writer, sheet_name="PAGOS2022", index=False)

# 10. Verificación
print("IDs asignados en PAGOS2022:")
print(df_target[["SOCIO", "MANZANA", df_target.columns[0], "ID"]].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Archivos disponibles en el directorio de Energía Eléctrica:
['COMERCIANTESVIGLIMP.xlsx', 'no_encontrados.xlsx', 'comerciantes.xlsx', 'OUTPUT', 'socios_no_reconocidos-2025.xlsx', 'ENERGIAELECTRICA20211.xlsx', 'ENERGIAELECTRICA2021_actualizado.xlsx', 'ENERGIAELECTRICA2021.xlsx']
Hojas en el archivo de energía eléctrica:
['MED.2025', 'PAGOS25', 'MED.2024', 'PAGOS2024', 'MED.2023', 'PAGOS2023', 'Sheet4', 'CONVENIOS', 'MED.2022', 'PAGOS2022', 'MED21', '2021', 'PAGOS-2020', 'PAGOS-2019', 'Sheet12', 'Sheet3', 'RECIBOS', 'Sin MED', 'Sheet11']
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2022-01",
        "monto": 112.5,
        "fecha_pago": "02/10/2022",
        "operacion": "3436"
    },
    {
        "id_comerciante": 529,
        "periodo": "2022-02",
        "monto": 124.5,
        "fecha_pago": "15/03/2022",
     

In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS25", header=0)
df_target = pd.read_excel(energy_path, sheet_name="MED2025", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df["MANZANA"] = df["MANZANA"].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref["MANZANA"] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target["MANZANA"] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAFAC2025.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS25", index=False)
    df_target.to_excel(writer, sheet_name="MED2025", index=False)

# 10. Verificación
print("IDs asignados en PAGOS2025:")
print(df_target[["SOCIO", "MANZANA", df_target.columns[0], "ID"]].head())


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


KeyError: 'MANZANA'

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS2025", header=0)
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]

# 9. Crear JSON de pagos para todos los comerciantes (sin agrupar)
json_list = []

cols = df_energy.columns.tolist()
for _, row in df_recognized.iterrows():
    cod = int(row["ID"])

    for i, col in enumerate(cols):
        col_upper = col.upper()
        for mes_nombre in meses:
            if mes_nombre in col_upper:
                monto = row[col]
                try:
                    emi_raw = row[cols[i+1]]  # FECHA
                    operacion = row[cols[i+2]]  # REC.
                except IndexError:
                    continue

                # Convertir la fecha de pago correctamente con mes/día/año (sin error)
                emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=False)

                if pd.isna(emi_date) or pd.isna(monto):
                    continue

                # Definir periodo
                mes_num = str(meses.index(mes_nombre) + 1).zfill(2)
                periodo = f"2025-{mes_num}"

                pago = {
                    "id_comerciante": cod,
                    "periodo": periodo,
                    "monto": float(monto),
                    "fecha_pago": emi_date.strftime("%m/%d/%Y"),  # Cambiado a mes/día/año
                    "operacion": str(operacion)
                }

                json_list.append(pago)

# 10. Guardar JSON (sin agrupar por ID)
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/energia-2025.json"
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 11. Mostrar solo los primeros 5 registros
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 12. Logs finales
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {energy_path}")


### **VIGILANCIAAAA**

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/COMERCIANTESVIGLIMP.xlsx"
comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="2025", header=0)
df_energy.columns = df_energy.columns.str.strip().str.upper()
df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
mapping = dict(zip(
    df_com["SOCIO"].astype(str).str.strip().str.upper(),
    df_com["id_comerciante"]
))

# 6. Rellenar ID donde esté vacío
df_energy["ID"] = df_energy["ID"].fillna(
    df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir meses
meses = [
    "ENERO", "FEBRERO", "MARZO", "ABRIL", "MAYO", "JUNIO",
    "JULIO", "AGOSTO", "SEPTIEMBRE", "OCTUBRE", "NOVIEMBRE", "DICIEMBRE"
]

# 9. Crear JSON de pagos
json_list = []
cols = df_energy.columns.tolist()

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    for i in range(len(cols)):
        col_upper = str(cols[i]).strip().upper()
        for mes_nombre in meses:
            if col_upper == mes_nombre:  # ✅ solo un `if`
                try:
                    monto_raw = row[cols[i]]
                    emi_raw = row[cols[i + 1]] if i + 1 < len(cols) else None
                    operacion_raw = row[cols[i + 2]] if i + 2 < len(cols) else ""
                except Exception:
                    continue

                # Validar que monto_raw no sea una Serie
                if isinstance(monto_raw, pd.Series) or isinstance(emi_raw, pd.Series):
                    continue

                try:
                    monto = float(monto_raw)
                except:
                    continue

                emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
                if pd.isna(monto) or pd.isna(emi_date):
                    continue

                mes_num = str(meses.index(mes_nombre) + 1).zfill(2)
                periodo = f"2025-{mes_num}"

                pago = {
                    "id_comerciante": cod,
                    "periodo": periodo,
                    "monto": monto,
                    "fecha_pago": emi_date.strftime("%d/%m/%Y"),
                    "operacion": str(operacion_raw)
                }

                json_list.append(pago)

# 10. Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "vigilancia-2025.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 11. Mostrar ejemplo
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 12. Logs
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 529,
        "periodo": "2025-02",
        "monto": 60.0,
        "fecha_pago": "04/03/2025",
        "operacion": "19457"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-03",
        "monto": 60.0,
        "fecha_pago": "26/03/2025",
        "operacion": "19741"
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-04",
        "monto": 60.0,
        "fecha_pago": "23/04/2025",
        "operacion": "20106"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-02",
        "monto": 60.0,
        "fecha_pago": "27/03/2025",
        "operacion": "19743"
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-03",
        "monto": 60.0,
        "fecha_pago": "16/04/2025",
        "operacion": "20024"
    },
    {
        "id_comercian

# PAGOS **VIGILANCIAAAAA**

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime
from openpyxl.utils import column_index_from_string

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/COMERCIANTESVIGLIMP.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="2019", header=0)
df_energy.columns = df_energy.columns.str.strip().str.upper()

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()

# 5. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 6. Definir mapeo de columnas para cada mes
mes_a_columnas = {
    "ENERO":     {"monto":"E",  "fecha":"F",  "oper":"G"},
    "FEBRERO":   {"monto":"H",  "fecha":"I",  "oper":"J"},
    "MARZO":     {"monto":"K",  "fecha":"L",  "oper":"M"},
    "ABRIL":     {"monto":"N",  "fecha":"O",  "oper":"P"},
    "MAYO":      {"monto":"Q",  "fecha":"R",  "oper":"S"},
    "JUNIO":     {"monto":"T",  "fecha":"U",  "oper":"V"},
    "JULIO":     {"monto":"W",  "fecha":"X",  "oper":"Y"},
    "AGOSTO":    {"monto":"Z",  "fecha":"AA", "oper":"AB"},
    "SEPTIEMBRE":{"monto":"AC", "fecha":"AD", "oper":"AE"},
    "OCTUBRE":   {"monto":"AF", "fecha":"AG", "oper":"AH"},
    "NOVIEMBRE": {"monto":"AI", "fecha":"AJ", "oper":"AK"},
    "DICIEMBRE": {"monto":"AM", "fecha":"AN", "oper":"AO"},
}

# 7. Convertir las letras de columnas a índices de pandas (0 basado)
for mes, cols in mes_a_columnas.items():
    for tipo, letra in cols.items():
        idx = column_index_from_string(letra) - 1
        mes_a_columnas[mes][tipo] = idx

# 8. Crear JSON de pagos
json_list = []

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    for mes, cols in mes_a_columnas.items():
        monto_raw = row.iloc[cols["monto"]]
        emi_raw   = row.iloc[cols["fecha"]]
        oper_raw  = row.iloc[cols["oper"]]

        try:
            monto = float(monto_raw)
        except:
            continue

        if pd.isna(monto) or monto <= 0:
            continue

        # Fecha: permitir nula y marcar como 'S/F' si falta
        emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
        fecha_pago = emi_date.strftime("%d/%m/%Y") if not pd.isna(emi_date) else "S/F"

        # Operación: marcar como 'S/O' si falta
        operacion = str(oper_raw).strip() if pd.notna(oper_raw) and str(oper_raw).strip() else "S/R"

        mes_num = str(list(mes_a_columnas).index(mes) + 1).zfill(2)
        periodo = f"2019-{mes_num}"

        pago = {
            "id_comerciante": cod,
            "periodo": periodo,
            "monto": monto,
            "fecha_pago": fecha_pago,
            "operacion": operacion
        }
        json_list.append(pago)

# 9. Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Vigilancia"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "vigilancia-2019.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 10. Mostrar ejemplo
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 11. Logs
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 684,
        "periodo": "2019-01",
        "monto": 40.0,
        "fecha_pago": "25/01/2019",
        "operacion": "3878"
    },
    {
        "id_comerciante": 684,
        "periodo": "2019-02",
        "monto": 40.0,
        "fecha_pago": "25/02/2019",
        "operacion": "3878"
    },
    {
        "id_comerciante": 684,
        "periodo": "2019-03",
        "monto": 40.0,
        "fecha_pago": "25/03/2019",
        "operacion": "3878"
    },
    {
        "id_comerciante": 684,
        "periodo": "2019-04",
        "monto": 40.0,
        "fecha_pago": "25/04/2019",
        "operacion": "3878"
    },
    {
        "id_comerciante": 684,
        "periodo": "2019-05",
        "monto": 40.0,
        "fecha_pago": "25/05/2019",
        "operacion": "3878"
    },
    {
        "id_comerciante"

### ***Facturacion de ENERGIA ELECTRICA***

> Codigo para crear obtener las facturas




In [ ]:
from google.colab import drive
from decimal import Decimal, ROUND_HALF_UP
import pandas as pd
import json
import os
import re

# Función para validar si un valor es numérico
def es_numero_valido(valor):
    try:
        return bool(re.match(r'^-?\d+(\.\d+)?$', str(valor).strip()))
    except:
        return False

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"

# 3. Leer la hoja MED2025
df_energy = pd.read_excel(energy_path, sheet_name="MED2025")

# 4. Normalizar nombres de columnas y limpiar datos
df_energy.columns = df_energy.columns.str.strip().str.upper()
df_energy["ID"] = pd.to_numeric(df_energy["ID"], errors="coerce")

# 5. Definir columnas por mes
columnas_mes = {
    "2025-01": "ENERO",
    "2025-02": "FEBRERO",
    "2025-03": "MARZO",
    "2025-04": "ABRIL",
    "2025-05": "MAYO",
    "2025-06": "JUNIA",
    "2025-07": "JULIO",
    "2025-08": "AGOSTO",
    "2025-09": "SEPTIEMBRE",
    "2025-10": "OCTUBRE",
    "2025-11": "NOVIEMBRE",
    "2025-12": "DICIEMBRE",
}

# 6. Crear lista para el JSON
json_list = []

for _, row in df_energy.iterrows():
    id_comerciante = row.get("ID", None)
    if pd.isna(id_comerciante):
        continue

    for periodo, nombre_mes in columnas_mes.items():
        if nombre_mes in df_energy.columns:
            monto = row.get(nombre_mes, None)

            try:
                if pd.notna(monto) and str(monto).strip() != "" and es_numero_valido(monto):
                    monto_decimal = Decimal(str(monto).strip())
                    monto_redondeado = float(monto_decimal.quantize(Decimal("0.1"), rounding=ROUND_HALF_UP))
                    if monto_redondeado > 0:
                        json_list.append({
                            "id_comerciante": int(id_comerciante),
                            "periodo": periodo,
                            "monto": monto_redondeado
                        })
            except (ValueError, TypeError, ArithmeticError):
                continue  # Saltar errores de conversión

# 7. Guardar el JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Energia"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "energia_2025_factura.json")

with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 8. Mostrar ejemplo
print("Primeros pagos exportados:")
print(json.dumps(json_list[:10], indent=4, ensure_ascii=False))
print(f"Total pagos generados: {len(json_list)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Primeros pagos exportados:
[
    {
        "id_comerciante": 529,
        "periodo": "2025-01",
        "monto": 116.6
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-02",
        "monto": 120.8
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-03",
        "monto": 135.6
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-04",
        "monto": 129.1
    },
    {
        "id_comerciante": 529,
        "periodo": "2025-05",
        "monto": 116.5
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-01",
        "monto": 34.5
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-02",
        "monto": 31.9
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-03",
        "monto": 35.9
    },
    {
        "id_comerciante": 18,
        "periodo": "2025-04",
      

### Medicion de energiaaaaaaaaaaaaaa

In [ ]:
from google.colab import drive
from decimal import Decimal, ROUND_HALF_UP, InvalidOperation
import pandas as pd
import json
import os

# Función auxiliar para convertir letra de columna a índice
def letra_a_indice(letra):
    letra = letra.upper()
    indice = 0
    for char in letra:
        indice = indice * 26 + (ord(char) - ord('A') + 1)
    return indice - 1  # Python indexa desde 0

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Leer archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"
df_energy = pd.read_excel(energy_path, sheet_name="MED2021")
print("Columnas originales:")
print(df_energy.columns.tolist())

# 3. Convertir a valores sin nombres (por índice)
df_energy_raw = df_energy.values

# 4. Asignación por letras de columnas
# Ejemplo: D = columna 3, E = 4, F = 5, G = 6 (empezando desde 0)
#meses_columnas = [
#    ("2025-01", "D", "E", "F", "G"),
#    ("2025-02", "E", "H", "I", "J"),
#    ("2025-03", "H", "K", "L", "M"),
#    ("2025-04", "K", "N", "O", "P"),
#    ("2025-05", "N", "Q", "R", "S"),
#    ("2025-06", "Q", "T", "U", "V"),
#    ("2025-07", "T", "W", "X", "Y"),
#    ("2025-08", "W", "Z", "AA", "AB"),
#    ("2025-09", "Z", "AC", "AD", "AE"),
#    ("2025-10", "AC", "AF", "AG", "AH"),
#    ("2025-11", "AF", "AI", "AJ", "AK"),
#    ("2025-12", "AI", "AL", "AM", "AN")
#]
meses_columnas = [
    ("2021-01", "G", "H", "I", "J"),
    ("2021-02", "H", "K", "L", "M"),
    ("2021-03", "K", "N", "O", "P"),
    ("2021-06", "N", "Q", "R", "S"),
    ("2021-07", "Q", "T", "U", "V"),
    ("2021-08", "T", "W", "X", "Y"),
    ("2021-09", "W", "Z", "AA", "AB"),
    ("2021-10", "Z", "AC", "AD", "AE"),
    ("2021-11", "AC", "AF", "AG", "AH"),
    ("2021-12", "AF", "AI", "AJ", "AK")
]

# 5. Generar JSON
json_list = []
for row in df_energy_raw:
    try:
        id_comerciante = int(row[letra_a_indice("AL")])  # ID en columna A
    except:
        continue

    for periodo, col_ant, col_act, col_watts, col_monto in meses_columnas:
        try:
            idx_ant = letra_a_indice(col_ant)
            idx_act = letra_a_indice(col_act)
            idx_watts = letra_a_indice(col_watts)
            idx_monto = letra_a_indice(col_monto)

            lectura_anterior = row[idx_ant]
            lectura_actual = row[idx_act]
            watts = row[idx_watts]
            monto = row[idx_monto]

            if pd.isna(lectura_anterior) or pd.isna(lectura_actual) or pd.isna(watts) or pd.isna(monto):
                continue

            lectura_anterior = float(lectura_anterior)
            lectura_actual = float(lectura_actual)
            watts = float(watts)
            #monto = float(Decimal(str(monto)).quantize(Decimal("0.1"), rounding=ROUND_HALF_UP))
            #monto = round(float(monto), 2)
            monto=(float(monto))

            json_list.append({
                "comerciante_id": id_comerciante,
                "mes": periodo,
                "lectura_anterior": lectura_anterior,
                "lectura_actual": lectura_actual,
                "watts": watts,
                "monto": monto
            })
        except Exception:
            continue

# 6. Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT//Energia/MEDICION"
os.makedirs(output_dir, exist_ok=True)
output_path = os.path.join(output_dir, "energia_2021_medicion.json")

with open(output_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 7. Mostrar ejemplo
print("\nEjemplo de registros exportados:")
print(json.dumps(json_list[:5], indent=4, ensure_ascii=False))
print(f"\nTotal registros exportados: {len(json_list)}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Columnas originales:
['                          ', 'PUESTO', 'SOCIO', 'M NOV', 'M DIC', 'CON EN', 'DICIEMBRE', 'M ENE', 'CON ENE', 'ENERO', 'M FEB', 'CON FEB', 'FEBRERO', 'M MAR', 'CON MAR', 'MARZO', 'M JUN', 'CON JUN', 'JUNIO', 'M JUL', 'CON JUL', 'JULIO', 'M AGOS', 'CON AGOS', 'AGOSTO', 'M SEP', 'CON SEP', 'SEPTIEMBRE', 'M OCT', 'CON OCT', 'OCTUBRE', 'M NOV.1', 'CON NOV', 'NOVIEMBRE', 'M DIC.1', 'CON DIC', 'DICIEMBRE.1', 'ID']

Ejemplo de registros exportados:
[
    {
        "comerciante_id": 529,
        "mes": "2021-01",
        "lectura_anterior": 225.28,
        "lectura_actual": 8522.1,
        "watts": 131.3,
        "monto": 113.04
    },
    {
        "comerciante_id": 529,
        "mes": "2021-02",
        "lectura_anterior": 8522.1,
        "lectura_actual": 8664.2,
        "watts": 142.1,
        "monto": 121.68
    },
    {
        "comerciant

PAGOS ENERGIA **ELECTRICA**

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime
from openpyxl.utils import column_index_from_string

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"
#comerciantes_path = "/content/drive/MyDrive/AcomemopV/Data/comerciantes.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS2021", header=0)
df_energy.columns = df_energy.columns.str.strip().str.upper()
#df_com = pd.read_excel(comerciantes_path, header=0)

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()
#df_com.columns = df_com.columns.str.strip()

# 5. Crear mapeo SOCIO → id_comerciante
#mapping = dict(zip(
 #   df_com["SOCIO"].astype(str).str.strip().str.upper(),
  #  df_com["id_comerciante"]
#))

# 6. Rellenar ID donde esté vacío
#df_energy["ID"] = df_energy["ID"].fillna(
 #   df_energy["SOCIO"].astype(str).str.strip().str.upper().map(mapping)
#)

# 7. Crear DataFrame de pagos reconocidos
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir mapeo de columnas para cada mes
mes_a_columnas = {
    "ENERO":     {"monto":"E",  "fecha":"F",  "oper":"G"},
    "FEBRERO":   {"monto":"H",  "fecha":"I",  "oper":"J"},
    "MARZO":     {"monto":"K",  "fecha":"L",  "oper":"M"},
    "ABRIL":     {"monto":"N",  "fecha":"O",  "oper":"P"},
    "MAYO":      {"monto":"Q",  "fecha":"R",  "oper":"S"},
    "JUNIO":     {"monto":"T",  "fecha":"U",  "oper":"V"},
    "JULIO":     {"monto":"W",  "fecha":"X",  "oper":"Y"},
    "AGOSTO":    {"monto":"Z",  "fecha":"AA", "oper":"AB"},
    "SEPTIEMBRE":{"monto":"AC", "fecha":"AD", "oper":"AE"},
    "OCTUBRE":   {"monto":"AF", "fecha":"AG", "oper":"AH"},
    "NOVIEMBRE": {"monto":"AI", "fecha":"AJ", "oper":"AK"},
    "DICIEMBRE": {"monto":"AL", "fecha":"AM", "oper":"AN"},
}

# 9. Convertir las letras de las columnas a índices de pandas (0 basado)
for mes, cols in mes_a_columnas.items():
    for tipo, letra in cols.items():
        # openpyxl da 1-based, pandas 0-based
        idx = column_index_from_string(letra) - 1
        mes_a_columnas[mes][tipo] = idx

# 10. Crear JSON de pagos
json_list = []

for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    for mes, cols in mes_a_columnas.items():
        monto_raw = row.iloc[cols["monto"]]
        emi_raw   = row.iloc[cols["fecha"]]
        oper_raw  = row.iloc[cols["oper"]]

        # Conversión y validación
        try:
            monto = float(monto_raw)
            emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
        except:
            continue
        if pd.isna(monto) or pd.isna(emi_date):
            continue

        mes_num = str(list(mes_a_columnas).index(mes) + 1).zfill(2)
        periodo = f"2021-{mes_num}"

        pago = {
            "id_comerciante": cod,
            "periodo": periodo,
            "monto": monto,
            "fecha_pago": emi_date.strftime("%d/%m/%Y"),
            "operacion": str(oper_raw)
        }
        json_list.append(pago)

# 11. Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Energia/PAGOS"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "energia-2021.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 12. Mostrar ejemplo
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 13. Logs
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


IndexError: single positional indexer is out-of-bounds

Para combinar ID------------------------------------------------------------------------------


In [ ]:
from google.colab import drive
import pandas as pd

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"

# 3. Leer las dos hojas
df_ref = pd.read_excel(energy_path, sheet_name="PAGOS2022", header=0)
df_target = pd.read_excel(energy_path, sheet_name="MED21", header=0)

# 4. Normalizar columnas
for df in [df_ref, df_target]:
    df.columns = df.columns.str.strip()
    df["SOCIO"] = df["SOCIO"].astype(str).str.strip().str.upper()
    df.iloc[:, 1] = df.iloc[:, 1].astype(str).str.strip().str.upper()
    df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)

# 5. Crear clave de combinación: LOTE + MANZANA + SOCIO
df_ref["CLAVE"] = df_ref.iloc[:, 0] + "|" + df_ref.iloc[:, 1] + "|" + df_ref["SOCIO"]
df_target["CLAVE"] = df_target.iloc[:, 0] + "|" + df_target.iloc[:, 1] + "|" + df_target["SOCIO"]

# 6. Crear diccionario de mapeo: CLAVE → ID
clave_id_map = df_ref.set_index("CLAVE")["ID"].dropna().to_dict()

# 7. Asignar el ID según coincidencia en clave
df_target["ID"] = df_target["CLAVE"].map(clave_id_map)

# 8. Eliminar columna auxiliar
df_ref.drop(columns=["CLAVE"], inplace=True)
df_target.drop(columns=["CLAVE"], inplace=True)

# 9. Guardar archivo actualizado
output_path = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/new.xlsx"
with pd.ExcelWriter(output_path, engine="openpyxl", mode="w") as writer:
    df_ref.to_excel(writer, sheet_name="PAGOS2022", index=False)
    df_target.to_excel(writer, sheet_name="MED21", index=False)

# 10. Verificación
print("IDs asignados en MED21:")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-4-d7c4886c0523>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
424    30.0
425    30.0
426     NAN
427     NAN
428    13.0
Name: , Length: 429, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna (LOTE)
<ipython-input-4-d7c4886c0523>:19: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise in a future error of pandas. Value '0       1.0
1       1.0
2       1.0
3       1.0
4       1.0
       ... 
422    30.0
423    30.0
424    30.0
425    30.0
426     NAN
Name: , Length: 427, dtype: object' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df.iloc[:, 0] = df.iloc[:, 0].astype(str).str.strip().str.upper()  # Primera columna

IDs asignados en MED21:


In [5]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime
from openpyxl.utils import column_index_from_string

# 1. Montar Drive
drive.mount('/content/drive')

# 2. Definir rutas
energy_path = "/content/drive/MyDrive/AcomemopV/Data/ENERGIAELECTRICA20211.xlsx"

# 3. Leer datos
df_energy = pd.read_excel(energy_path, sheet_name="PAGOS2021", header=0)
df_energy.columns = df_energy.columns.str.strip().str.upper()
print(df_energy.head(20))

# 4. Normalizar columnas
df_energy.columns = df_energy.columns.str.strip()

# 7. Crear DataFrame de pagos reconocidos (omitido el mapeo del ID para mantener tu versión actual)
df_recognized = df_energy.dropna(subset=["ID"])

# 8. Definir mapeo de columnas para cada mes
mes_a_columnas = {
    "ENERO":     {"monto":"E",  "fecha":"F",  "oper":"G"},
    "FEBRERO":   {"monto":"H",  "fecha":"I",  "oper":"J"},
    "MARZO":     {"monto":"K",  "fecha":"L",  "oper":"M"},
    "ABRIL":     {"monto":"K",  "fecha":"L",  "oper":"M"},
    "MAYO":      {"monto":"N",  "fecha":"O",  "oper":"P"},
    "JUNIO":     {"monto":"Q",  "fecha":"R",  "oper":"S"},
    "JULIO":     {"monto":"T",  "fecha":"U",  "oper":"V"},
    "AGOSTO":    {"monto":"W",  "fecha":"X",  "oper":"Y"},
    "SEPTIEMBRE":{"monto":"Z",  "fecha":"AA", "oper":"AB"},
    "OCTUBRE":   {"monto":"AC", "fecha":"AD", "oper":"AE"},
    "NOVIEMBRE": {"monto":"AF", "fecha":"AG", "oper":"AH"},
    "DICIEMBRE": {"monto":"AI", "fecha":"AJ", "oper":"AK"}
}

# 9. Convertir letras a índices
for mes, cols in mes_a_columnas.items():
    for tipo, letra in cols.items():
        idx = column_index_from_string(letra) - 1
        mes_a_columnas[mes][tipo] = idx

# 10. Crear JSON de pagos
json_list = []
for _, row in df_recognized.iterrows():
    cod = int(row["ID"])
    for mes, cols in mes_a_columnas.items():
        monto_raw = row.iloc[cols["monto"]]
        emi_raw   = row.iloc[cols["fecha"]]
        oper_raw  = row.iloc[cols["oper"]]

        try:
            monto = round(float(monto_raw), 2)
        except:
            continue
        if pd.isna(monto):
            continue

        mes_num = str(list(mes_a_columnas).index(mes) + 1).zfill(2)
        anio = "2021"
        emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)

        # Si la fecha es NaN, usar el 25 del mes
        if pd.isna(emi_date):
            emi_date = pd.to_datetime(f"{anio}-{mes_num}-25")

        pago = {
            "id_comerciante": cod,
            "periodo": f"{anio}-{mes_num}",
            "monto": monto,
            "fecha_pago": emi_date.strftime("%d/%m/%Y"),
            "operacion": str(oper_raw)
        }
        json_list.append(pago)

# 11. Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Energia/PAGOS"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "energia-2021.json")
with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 12. Mostrar ejemplo
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

# 13. Logs
print("Total registros RECONOCIDOS (con ID):", len(df_recognized))
print(f"Archivo actualizado: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
        MANZANA                               SOCIO     ID   ENERO  FECHA  \
0   1.0      12           SILVA SALDARRIAGA ALBERTO  529.0  113.04    NaN   
1   1.0     13A           CARRASCO CASTILLO ALFONSO   18.0   46.16    NaN   
2   1.0     13B           VILLEGAS PACHERREZ MANUEL  621.0   19.76    NaN   
3   1.0    14-B   HIDALGO VDA. DE CHAPARRO HILBERTA  655.0       8    NaN   
4   1.0    14-A   HIDALGO VDA. DE CHAPARRO HILBERTA   19.0     NaN    NaN   
5   1.0      15                LEDESMA FLORES JORGE   20.0   33.76    NaN   
6   1.0      16                    PAIVA VITE JUAN    21.0   45.76    NaN   
7   1.0    17-A          ANCAJIMA GOYCOCHEA EDGARDO  627.0   41.36    NaN   
8   1.0    17-B          ANCAJIMA GOYCOCHEA EDGARDO  657.0     NaN    NaN   
9   1.0      18                LACHIRA LEON ROLANDO   23.0       8    NaN   
10  1.0      19         

## PAGOSSSSSS DE AGUAAAAAAAAAAAAAAAAAAAAAA

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime
from openpyxl.utils import column_index_from_string

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo Excel
energy_path = "/content/drive/MyDrive/AcomemopV/Data/deudaagua.xlsx"

# 3. Leer hoja específica del Excel (con encabezado en fila 2)
df_energy = pd.read_excel(energy_path, sheet_name="AGUA-2022", header=1)
df_energy.columns = df_energy.columns.str.strip().str.upper()  # Estandarizar nombres de columnas
df_energy.columns = df_energy.columns.str.strip()

# 4. Normalizar columna SERVICIO y filtrar sólo registros válidos (agua o desague)
df_energy["SERVICIO"] = df_energy["SERVICIO"].astype(str).str.strip().str.lower()
df_valid_service = df_energy[df_energy["SERVICIO"].isin(["agua", "desague"])]

# 5. Filtrar registros que tienen ID de comerciante (RECONOCIDOS) y servicio válido
df_recognized = df_valid_service.dropna(subset=["ID"])

# 6. Mapeo de columnas por mes
mes_a_columnas = {
    "ENERO":     {"monto":"G",  "fecha":"H",  "oper":"I"},
    "FEBRERO":   {"monto":"J",  "fecha":"K",  "oper":"L"},
    "MARZO":     {"monto":"M",  "fecha":"N",  "oper":"O"},
    "ABRIL":     {"monto":"P",  "fecha":"Q",  "oper":"R"},
    "MAYO":      {"monto":"S",  "fecha":"T",  "oper":"U"},
    "JUNIO":     {"monto":"V",  "fecha":"W",  "oper":"X"},
    "JULIO":     {"monto":"Y",  "fecha":"Z",  "oper":"AA"},
    "AGOSTO":    {"monto":"AB", "fecha":"AC", "oper":"AD"},
    "SEPTIEMBRE":{"monto":"AE", "fecha":"AF", "oper":"AG"},
    "OCTUBRE":   {"monto":"AH", "fecha":"AI", "oper":"AJ"},
    "NOVIEMBRE": {"monto":"AK", "fecha":"AL", "oper":"AM"},
    "DICIEMBRE": {"monto":"AN", "fecha":"AO", "oper":"AP"},
}

# 7. Convertir letras de columnas a índices (0-based para pandas)
for mes, columnas in mes_a_columnas.items():
    for tipo, letra in columnas.items():
        idx = column_index_from_string(letra) - 1  # Ajustar a índice 0-based
        mes_a_columnas[mes][tipo] = idx

# 8. Construcción de lista JSON de pagos
json_list = []

for _, row in df_recognized.iterrows():
    try:
        cod = int(row["ID"])
    except:
        continue  # Saltar si ID no es convertible a int

    for mes, cols in mes_a_columnas.items():
        monto_raw = row.iloc[cols["monto"]]
        emi_raw   = row.iloc[cols["fecha"]]
        oper_raw  = row.iloc[cols["oper"]]

        # Validación de datos
        try:
            monto = float(monto_raw)
            emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
        except:
            continue
        if pd.isna(monto) or pd.isna(emi_date):
            continue

        mes_num = str(list(mes_a_columnas).index(mes) + 1).zfill(2)
        periodo = f"2022-{mes_num}"

        pago = {
            "id_comerciante": cod,
            "periodo": periodo,
            "monto": monto,
            "fecha_pago": emi_date.strftime("%d/%m/%Y"),
            "operacion": str(oper_raw),
            "servicio": row["SERVICIO"]
        }
        json_list.append(pago)

# 9. Guardar archivo JSON en Drive
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Agua"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "agua-2022.json")

with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# 10. Mostrar ejemplos y log final
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

print("\nResumen:")
print("Total registros RECONOCIDOS (con ID y servicio válido):", len(df_recognized))
print(f"Archivo JSON generado correctamente en: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 53,
        "periodo": "2022-01",
        "monto": 100.0,
        "fecha_pago": "16/02/2022",
        "operacion": "51",
        "servicio": "agua"
    },
    {
        "id_comerciante": 53,
        "periodo": "2022-02",
        "monto": 100.0,
        "fecha_pago": "11/03/2022",
        "operacion": "61",
        "servicio": "agua"
    },
    {
        "id_comerciante": 53,
        "periodo": "2022-03",
        "monto": 100.0,
        "fecha_pago": "11/03/2022",
        "operacion": "62.0",
        "servicio": "agua"
    },
    {
        "id_comerciante": 53,
        "periodo": "2022-04",
        "monto": 100.0,
        "fecha_pago": "18/05/2022",
        "operacion": "111",
        "servicio": "agua"
    },
    {
        "id_comerciante": 53,
        "periodo": "2022-05",
        "monto": 1

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime
from openpyxl.utils import column_index_from_string

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo Excel
energy_path = "/content/drive/MyDrive/AcomemopV/Data/deudaagua.xlsx"

# 3. Leer hoja específica del Excel (con encabezado en fila 2)
df_energy = pd.read_excel(energy_path, sheet_name="AGUA-2025", header=1)
df_energy.columns = df_energy.columns.str.strip().str.upper()  # Estandarizar nombres de columnas

# 4. Normalizar columna SERVICIO y filtrar sólo registros válidos (agua o desague)
df_energy["SERVICIO"] = df_energy["SERVICIO"].astype(str).str.strip().str.lower()
df_valid_service = df_energy[df_energy["SERVICIO"].isin(["agua", "desague"])]

# 5. Filtrar registros con ID (reconocidos)
df_recognized = df_valid_service.dropna(subset=["ID"])

# 6. Mapeo de columnas por mes
mes_a_columnas = {
    "ENERO":     {"monto":"G",  "fecha":"H",  "oper":"I"},
    "FEBRERO":   {"monto":"J",  "fecha":"K",  "oper":"L"},
    "MARZO":     {"monto":"M",  "fecha":"N",  "oper":"O"},
    "ABRIL":     {"monto":"P",  "fecha":"Q",  "oper":"R"},
    "MAYO":      {"monto":"S",  "fecha":"T",  "oper":"U"},
    "JUNIO":     {"monto":"V",  "fecha":"W",  "oper":"X"},
    "JULIO":     {"monto":"Y",  "fecha":"Z",  "oper":"AA"},
    "AGOSTO":    {"monto":"AB", "fecha":"AC", "oper":"AD"},
    "SEPTIEMBRE":{"monto":"AE", "fecha":"AF", "oper":"AG"},
    "OCTUBRE":   {"monto":"AH", "fecha":"AI", "oper":"AJ"},
    "NOVIEMBRE": {"monto":"AK", "fecha":"AL", "oper":"AM"},
    "DICIEMBRE": {"monto":"AN", "fecha":"AO", "oper":"AP"},
}

# Convertir letras de columnas a índices (0-based para pandas)
for mes, columnas in mes_a_columnas.items():
    for tipo, letra in columnas.items():
        mes_a_columnas[mes][tipo] = column_index_from_string(letra) - 1

# 7. Construir lista de pagos
json_list = []

for cod, grupo in df_recognized.groupby("ID"):
    for mes_idx, mes in enumerate(mes_a_columnas.keys(), start=1):
        periodo = f"2025-{str(mes_idx).zfill(2)}"
        cols = mes_a_columnas[mes]

        for servicio in ["agua", "desague"]:
            fila = grupo[grupo["SERVICIO"] == servicio]
            if fila.empty:
                continue

            row = fila.iloc[0]

            monto_raw = row.iloc[cols["monto"]]
            emi_raw = row.iloc[cols["fecha"]]
            oper_raw = row.iloc[cols["oper"]]
            # Forzar que sea texto y mantener todo tal cual
            operacion_str = str(oper_raw).strip() if not pd.isna(oper_raw) else ""
            # Limpiar texto del monto
            monto_str = str(monto_raw).replace("S/.", "").replace("S/", "").replace(" ", "").strip().lower()

            # Ignorar texto como "sin consumo", vacío o inválido
            if monto_str == "" or "sinconsumo" in monto_str:
                continue

            try:
                monto = float(monto_str)
                emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
            except:
                continue

            if pd.isna(monto) or pd.isna(emi_date):
                continue

            pago = {
                "id_comerciante": int(cod),
                "periodo": periodo,
                "monto": monto,
                "fecha_pago": emi_date.strftime("%d/%m/%Y"),
                "operacion": operacion_str,
                "servicio": servicio
            }
            json_list.append(pago)

# Guardar JSON
output_dir = "/content/drive/MyDrive/AcomemopV/Data/OUTPUT/Agua"
os.makedirs(output_dir, exist_ok=True)
output_json_path = os.path.join(output_dir, "agua-2025.json")

with open(output_json_path, "w", encoding="utf-8") as f:
    json.dump(json_list, f, indent=4, ensure_ascii=False)

# Mostrar resumen
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

print("\nResumen:")
print("Total registros RECONOCIDOS (con ID y servicio válido):", len(df_recognized))
print(f"Archivo JSON generado correctamente en: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 53,
        "periodo": "2025-01",
        "monto": 100.0,
        "fecha_pago": "18/02/2025",
        "operacion": "942",
        "servicio": "agua"
    },
    {
        "id_comerciante": 53,
        "periodo": "2025-01",
        "monto": 50.0,
        "fecha_pago": "18/02/2025",
        "operacion": "942",
        "servicio": "desague"
    },
    {
        "id_comerciante": 53,
        "periodo": "2025-02",
        "monto": 100.0,
        "fecha_pago": "03/03/2025",
        "operacion": "960",
        "servicio": "agua"
    },
    {
        "id_comerciante": 53,
        "periodo": "2025-02",
        "monto": 50.0,
        "fecha_pago": "03/03/2025",
        "operacion": "960",
        "servicio": "desague"
    },
    {
        "id_comerciante": 53,
        "periodo": "2025-03",
        "mont

Facturas AGUAAAAAAAAAAAAAAAAA

In [ ]:
from google.colab import drive
import pandas as pd
import json
import os
from datetime import datetime
from openpyxl.utils import column_index_from_string

# 1. Montar Google Drive
drive.mount('/content/drive')

# 2. Definir ruta del archivo Excel
energy_path = "/content/drive/MyDrive/AcomemopV/Data/deudaagua.xlsx"

# 3. Leer hoja específica del Excel (con encabezado en fila 2)
df_energy = pd.read_excel(energy_path, sheet_name="AGUA-2022", header=1)
df_energy.columns = df_energy.columns.str.strip().str.upper()  # Estandarizar nombres de columnas

# 4. Normalizar columna SERVICIO y filtrar sólo registros válidos (agua o desague)
df_energy["SERVICIO"] = df_energy["SERVICIO"].astype(str).str.strip().str.lower()
df_valid_service = df_energy[df_energy["SERVICIO"].isin(["agua", "desague"])]

# 5. Filtrar registros con ID (reconocidos)
df_recognized = df_valid_service.dropna(subset=["ID"])

# 6. Mapeo de columnas por mes
mes_a_columnas = {
    "ENERO":     {"monto":"G",  "fecha":"H",  "oper":"I"},
    "FEBRERO":   {"monto":"J",  "fecha":"K",  "oper":"L"},
    "MARZO":     {"monto":"M",  "fecha":"N",  "oper":"O"},
    "ABRIL":     {"monto":"P",  "fecha":"Q",  "oper":"R"},
    "MAYO":      {"monto":"S",  "fecha":"T",  "oper":"U"},
    "JUNIO":     {"monto":"V",  "fecha":"W",  "oper":"X"},
    "JULIO":     {"monto":"Y",  "fecha":"Z",  "oper":"AA"},
    "AGOSTO":    {"monto":"AB", "fecha":"AC", "oper":"AD"},
    "SEPTIEMBRE":{"monto":"AE", "fecha":"AF", "oper":"AG"},
    "OCTUBRE":   {"monto":"AH", "fecha":"AI", "oper":"AJ"},
    "NOVIEMBRE": {"monto":"AK", "fecha":"AL", "oper":"AM"},
    "DICIEMBRE": {"monto":"AN", "fecha":"AO", "oper":"AP"},
}

# Convertir letras de columnas a índices (0-based para pandas)
for mes, columnas in mes_a_columnas.items():
    for tipo, letra in columnas.items():
        mes_a_columnas[mes][tipo] = column_index_from_string(letra) - 1

# 7. Construir lista de pagos
json_list = []

for cod, grupo in df_recognized.groupby("ID"):
    for mes_idx, mes in enumerate(mes_a_columnas.keys(), start=1):
        periodo = f"2022-{str(mes_idx).zfill(2)}"
        cols = mes_a_columnas[mes]

        for servicio in ["agua", "desague"]:
            fila = grupo[grupo["SERVICIO"] == servicio]
            if fila.empty:
                continue

            row = fila.iloc[0]

            monto_raw = row.iloc[cols["monto"]]
            emi_raw = row.iloc[cols["fecha"]]
            oper_raw = row.iloc[cols["oper"]]
            # Forzar que sea texto y mantener todo tal cual
            operacion_str = str(oper_raw).strip() if not pd.isna(oper_raw) else ""
            # Limpiar texto del monto
            monto_str = str(monto_raw).replace("S/.", "").replace("S/", "").replace(" ", "").strip().lower()

            # Ignorar texto como "sin consumo", vacío o inválido
            if monto_str == "" or "sinconsumo" in monto_str:
                continue

            try:
                monto = float(monto_str)
                emi_date = pd.to_datetime(emi_raw, errors="coerce", dayfirst=True)
            except:
                continue

            if pd.isna(monto) or pd.isna(emi_date):
                continue

            pago = {
                "id_comerciante": int(cod),
                "periodo": periodo,
                "monto": monto,
                "fecha_pago": emi_date.strftime("%d/%m/%Y"),
                "operacion": operacion_str,
                "servicio": servicio
            }
            json_list.append(pago)
# 8. Asegurar que cada comerciante tenga 2025-01 a 2025-06 para cada servicio

# Convertir a DataFrame para facilitar manejo
df_pagos = pd.DataFrame(json_list)

# Crear todos los periodos requeridos
periodos_faltantes = [f"2022-{str(m).zfill(2)}" for m in range(1, 13)]

# Agrupar por comerciante y servicio
comerciantes_servicios = df_recognized[["ID", "SERVICIO"]].drop_duplicates()

for _, row in comerciantes_servicios.iterrows():
    cod = int(row["ID"])
    servicio = row["SERVICIO"]

    pagos_comerciante = df_pagos[(df_pagos["id_comerciante"] == cod) & (df_pagos["servicio"] == servicio)]

    # Sacar monto de referencia (puede ser promedio o valor más frecuente)
    if not pagos_comerciante.empty:
        monto_referencia = pagos_comerciante["monto"].mode().iloc[0]  # Usa el valor más común
    else:
        # En caso no haya ninguno registrado, buscar directamente en df_recognized
        monto_referencia = None
        grupo = df_recognized[(df_recognized["ID"] == cod) & (df_recognized["SERVICIO"] == servicio)]
        for mes in mes_a_columnas.values():
            raw = grupo.iloc[0, mes["monto"]]
            if pd.notna(raw):
                try:
                    monto_str = str(raw).replace("S/.", "").replace("S/", "").replace(" ", "").strip().lower()
                    if monto_str != "" and "sinconsumo" not in monto_str:
                        monto_referencia = float(monto_str)
                        break
                except:
                    continue
        if monto_referencia is None:
            continue  # No hay forma de obtener un monto base

    # Buscar qué periodos faltan
    periodos_actuales = pagos_comerciante["periodo"].tolist()
    faltantes = set(periodos_faltantes) - set(periodos_actuales)

    for periodo in faltantes:
        pago = {
            "id_comerciante": cod,
            "periodo": periodo,
            "monto": monto_referencia,
            "fecha_pago": "",
            "operacion": "",
            "servicio": servicio
        }
        json_list.append(pago)

# Separar completas e incompletas
facturas_completas = [p for p in json_list if p["fecha_pago"] and p["operacion"]]
facturas_incompletas = [p for p in json_list if not p["fecha_pago"] or not p["operacion"]]

# Guardar JSONs separados
output_json_C = os.path.join(output_dir, "factura-agua-2022-C.json")
output_json_I = os.path.join(output_dir, "factura-agua-2022-I.json")

with open(output_json_C, "w", encoding="utf-8") as f:
    json.dump(facturas_completas, f, indent=4, ensure_ascii=False)

with open(output_json_I, "w", encoding="utf-8") as f:
    json.dump(facturas_incompletas, f, indent=4, ensure_ascii=False)

print("\nArchivos separados generados:")
print("✔ Completas:", output_json_C)
print("✔ Incompletas:", output_json_I)


# Mostrar resumen
print("Ejemplo de los primeros 12 pagos:")
print(json.dumps(json_list[:12], indent=4, ensure_ascii=False))

print("\nResumen:")
print("Total registros RECONOCIDOS (con ID y servicio válido):", len(df_recognized))
print(f"Archivo JSON generado correctamente en: {output_json_path}")


Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).

Archivos separados generados:
✔ Completas: /content/drive/MyDrive/AcomemopV/Data/OUTPUT/Agua/factura-agua-2022-C.json
✔ Incompletas: /content/drive/MyDrive/AcomemopV/Data/OUTPUT/Agua/factura-agua-2022-I.json
Ejemplo de los primeros 12 pagos:
[
    {
        "id_comerciante": 35,
        "periodo": "2022-01",
        "monto": 100.0,
        "fecha_pago": "19/03/2022",
        "operacion": "72",
        "servicio": "agua"
    },
    {
        "id_comerciante": 35,
        "periodo": "2022-01",
        "monto": 50.0,
        "fecha_pago": "19/03/2022",
        "operacion": "72",
        "servicio": "desague"
    },
    {
        "id_comerciante": 35,
        "periodo": "2022-02",
        "monto": 100.0,
        "fecha_pago": "19/03/2022",
        "operacion": "73",
        "servicio": "agua"
    },
    {
        "id_comerciante": 35,
        "periodo": "2022-02

In [ ]:

import requests
import time
import random

form_url = "https://docs.google.com/forms/d/e/1FAIpQLSe17m3vTxzZgtoVfCtayAE2kJnfTE6kC5x-4Tjde0zkPsNprA/formResponse"

field_ids = [
    "entry.324809467",
    "entry.1604326936",
    "entry.1758187744",
    "entry.251961348",
    "entry.1726222051",
    "entry.146779256",
    "entry.2012517417",
    "entry.592279989",
    "entry.2036908067",
    "entry.1704920714"
]

# Para simplificar, todas las preguntas son Sí/No
opciones = ["Sí", "No"]

num_respuestas = 50

for i in range(num_respuestas):
    respuesta = {}
    for field_id in field_ids:
        # Puedes personalizar la probabilidad, aquí ejemplo general
        respuesta[field_id] = random.choice(["No"] * 8 + ["Sí"] * 2)

    response = requests.post(form_url, data=respuesta)
    print(f"Enviada respuesta {i + 1}: {respuesta} - Estado HTTP {response.status_code}")

    time.sleep(random.uniform(1, 3))  # Evitar detección como bot


Enviada respuesta 1: {'entry.324809467': 'No', 'entry.1604326936': 'No', 'entry.1758187744': 'No', 'entry.251961348': 'No', 'entry.1726222051': 'No', 'entry.146779256': 'No', 'entry.2012517417': 'No', 'entry.592279989': 'Sí', 'entry.2036908067': 'No', 'entry.1704920714': 'No'} - Estado HTTP 200
Enviada respuesta 2: {'entry.324809467': 'No', 'entry.1604326936': 'Sí', 'entry.1758187744': 'No', 'entry.251961348': 'No', 'entry.1726222051': 'No', 'entry.146779256': 'No', 'entry.2012517417': 'No', 'entry.592279989': 'No', 'entry.2036908067': 'No', 'entry.1704920714': 'No'} - Estado HTTP 200
Enviada respuesta 3: {'entry.324809467': 'No', 'entry.1604326936': 'No', 'entry.1758187744': 'Sí', 'entry.251961348': 'No', 'entry.1726222051': 'No', 'entry.146779256': 'No', 'entry.2012517417': 'No', 'entry.592279989': 'No', 'entry.2036908067': 'Sí', 'entry.1704920714': 'No'} - Estado HTTP 200
Enviada respuesta 4: {'entry.324809467': 'No', 'entry.1604326936': 'No', 'entry.1758187744': 'Sí', 'entry.251961

In [ ]:
import requests
import time
import random

form_url = "https://docs.google.com/forms/d/e/1FAIpQLSe17m3vTxzZgtoVfCtayAE2kJnfTE6kC5x-4Tjde0zkPsNprA/formResponse"

field_ids = [
    "entry.324809467",    # 1
    "entry.1604326936",   # 2
    "entry.1758187744",   # 3 ✔️
    "entry.251961348",    # 4
    "entry.1726222051",   # 5 ✔️
    "entry.146779256",    # 6 ✔️
    "entry.2012517417",   # 7 ✔️
    "entry.592279989",    # 8 ✔️
    "entry.2036908067",   # 9
    "entry.1704920714"    # 10
]

# Índices de preguntas con mayor probabilidad de "Sí"
preguntas_mayoria_si = [2, 4, 5, 6, 7]  # Índices de la lista (basado en 0)

num_respuestas = 117

for i in range(num_respuestas):
    respuesta = {}
    for idx, field_id in enumerate(field_ids):
        if idx in preguntas_mayoria_si:
            respuesta[field_id] = random.choices(["Sí", "No"], weights=[80, 20])[0]
        else:
            respuesta[field_id] = random.choice(["Sí", "No"])

    response = requests.post(form_url, data=respuesta)
    print(f"Enviada respuesta {i + 1}: {respuesta} - Estado HTTP {response.status_code}")

    time.sleep(random.uniform(1, 3))  # Evitar detección como bot


Enviada respuesta 1: {'entry.324809467': 'No', 'entry.1604326936': 'Sí', 'entry.1758187744': 'Sí', 'entry.251961348': 'No', 'entry.1726222051': 'No', 'entry.146779256': 'No', 'entry.2012517417': 'Sí', 'entry.592279989': 'No', 'entry.2036908067': 'Sí', 'entry.1704920714': 'No'} - Estado HTTP 200
Enviada respuesta 2: {'entry.324809467': 'No', 'entry.1604326936': 'Sí', 'entry.1758187744': 'Sí', 'entry.251961348': 'Sí', 'entry.1726222051': 'Sí', 'entry.146779256': 'Sí', 'entry.2012517417': 'Sí', 'entry.592279989': 'Sí', 'entry.2036908067': 'Sí', 'entry.1704920714': 'No'} - Estado HTTP 200
Enviada respuesta 3: {'entry.324809467': 'Sí', 'entry.1604326936': 'Sí', 'entry.1758187744': 'Sí', 'entry.251961348': 'No', 'entry.1726222051': 'No', 'entry.146779256': 'Sí', 'entry.2012517417': 'Sí', 'entry.592279989': 'Sí', 'entry.2036908067': 'Sí', 'entry.1704920714': 'Sí'} - Estado HTTP 200
Enviada respuesta 4: {'entry.324809467': 'No', 'entry.1604326936': 'Sí', 'entry.1758187744': 'No', 'entry.251961